In [3]:
#|default_exp utils

# Utils

In [10]:
#|export
import os
import re
import numpy as np
from fastcore.all import *
from tsai.data.preparation import SlidingWindow

In [4]:
#|export
def convert_uuids_to_indices():
    cuda_visible_devices = os.environ.get("CUDA_VISIBLE_DEVICES", "")
    uuids = re.findall(r"\b[0-9a-fA-F]{8}(?:-[0-9a-fA-F]{4}){3}-[0-9a-fA-F]{12}\b", cuda_visible_devices)

    if uuids:
        indices = [str(i) for i in range(len(uuids))]
        os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(indices)

In [5]:
#|export
def percent_rel_error_(N, N_mc, N_mc_t0):
    """
    Return the percentage relative error with respect to the MC simulation
    See Eq. 23 of the paper by Giudici et al. (2023), 'Space debris density 
    propagation through a vinite volume method'
    Input:
        N: Predicted number of in-orbit elements over time
        N_mc: Number of in-orbit elements from the MC simulation over time
        N_mc_t0: Number of in-orbit elements from the MC simulation at t0
    Output:
        The percentage relative error (float)
    """
    return 100 * np.abs(N_mc - N) / N_mc_t0

In [6]:
# Test
N = np.array([1, 2, 3, 4, 5])
N_mc = np.array([1, 2, 3, 4, 5])
N_mc_t0 = N_mc[0]
res = percent_rel_error_(N, N_mc, N_mc_t0)
test_eq(res, np.zeros_like(res))

# Test 2
N = np.array([1, 2, 3, 4, 5])
N_mc = np.array([2, 4, 6, 8, 10])
N_mc_t0 = N_mc[0]
res = percent_rel_error_(N, N_mc, N_mc_t0)
test_eq(res, np.array([ 50., 100., 150., 200., 250.]))



In [23]:
#|export
@delegates(SlidingWindow)
def apply_sliding_window_3d(data, window_len, **kwargs):
    """
        Apply sliding window to 3D data. The data is assumed to have the shape
        (n_samples, n_features, n_time_steps).
        Input:
            data: 3D array
            window_len: int, length of the sliding window
            horizon: int, number of time steps to predict
            kwargs: additional arguments to SlidingWindow
        Output:
            X: 3D array, shape (n_samples, n_features, window_len)
            y: 3D array, shape (n_samples, n_features, horizon)
    """
    sw = SlidingWindow(window_len=window_len, **kwargs)
    # Apply the sliding window to every sample (loop on dimension 0) of the data
    slices = [sw(x) for x in data]
    # Concatenate the slices into single arrays (X and y)
    X = np.concatenate([s[0] for s in slices])
    y = np.concatenate([s[1] for s in slices])
    # In case y is a 2D array (config.horizon = 1), add the time dimension at the end
    if y.ndim == 2:
        y = np.expand_dims(y, -1)
    return X, y

In [25]:
# Test
data = np.random.rand(100, 3, 10)
X, y = apply_sliding_window_3d(data, window_len=5, horizon=1)
test_eq(X.shape, (90, 3, 5))

ValueError: zero-dimensional arrays cannot be concatenated

In [11]:
#|eval: false
#|hide
from nbdev import *
nbdev_export()